# Load and Process Graph Data

In this session, you will learn:

* Load graph data stored in CSV files.
* Construct a graph in DGL.
* Query structural information of a DGL graph.
* Load and pre-process node and edge features.


## Load graph data from CSV

Comma Separated Values (CSV) is a widely-used format for storing relational data. In this tutorial, we have prepared two CSV files that store [the Zachery's Karate Club network](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).
* The `nodes.csv` stores every club members and their attributes.
* The `edges.csv` stores the pair-wise interactions between two club members.

In [1]:
!ls -lh 'data'

total 24
-rw-r--r--  1 thchang  110265205   3.7K Aug 23 21:48 edges.csv
-rw-r--r--  1 thchang  110265205   1.2K Aug 23 21:48 gen_data.py
-rw-r--r--  1 thchang  110265205   461B Aug 23 21:48 nodes.csv


We can use `pandas` to load the two CSV files.

In [1]:
import pandas as pd

nodes_data = pd.read_csv('data/nodes.csv')
print(nodes_data)

    Id     Club  Age
0    0   Mr. Hi   45
1    1   Mr. Hi   33
2    2   Mr. Hi   36
3    3   Mr. Hi   31
4    4   Mr. Hi   41
5    5   Mr. Hi   42
6    6   Mr. Hi   48
7    7   Mr. Hi   41
8    8   Mr. Hi   30
9    9  Officer   35
10  10   Mr. Hi   38
11  11   Mr. Hi   44
12  12   Mr. Hi   37
13  13   Mr. Hi   39
14  14  Officer   36
15  15  Officer   38
16  16   Mr. Hi   47
17  17   Mr. Hi   45
18  18  Officer   41
19  19   Mr. Hi   31
20  20  Officer   31
21  21   Mr. Hi   44
22  22  Officer   42
23  23  Officer   32
24  24  Officer   30
25  25  Officer   50
26  26  Officer   30
27  27  Officer   43
28  28  Officer   48
29  29  Officer   40
30  30  Officer   39
31  31  Officer   45
32  32  Officer   47
33  33  Officer   33


In [2]:
edges_data = pd.read_csv('data/edges.csv')
print(edges_data)

     Src  Dst    Weight
0      0    1  0.318451
1      0    2  0.551215
2      0    3  0.227416
3      0    4  0.266919
4      0    5  0.475449
5      0    6  0.886263
6      0    7  0.160426
7      0    8  0.745981
8      0   10  0.589290
9      0   11  0.478159
10     0   12  0.782471
11     0   13  0.417957
12     0   17  0.368536
13     0   19  0.955193
14     0   21  0.961357
15     0   31  0.726227
16     1    0  0.648110
17     1    2  0.378915
18     1    3  0.243639
19     1    7  0.414836
20     1   13  0.615521
21     1   17  0.268028
22     1   19  0.112954
23     1   21  0.085523
24     1   30  0.593566
25     2    0  0.536214
26     2    1  0.576811
27     2    3  0.437126
28     2    7  0.130788
29     2    8  0.080224
..   ...  ...       ...
126   31   33  0.459360
127   32    2  0.883918
128   32    8  0.553993
129   32   14  0.419707
130   32   15  0.816858
131   32   18  0.304816
132   32   20  0.072799
133   32   22  0.397862
134   32   23  0.206909
135   32   29  0

We then construct a graph where each node is a club member and each edge represents their interactions. In DGL, **nodes are consecutive integers starting from zero**. Thus, when preparing the data, it is important to re-label or re-shuffle the row order so that the first row corresponding to the first nodes, so on and so forth.

In this example, we have already prepared the data in the correct order, so we can create the graph by the `'Src'` and `'Dst'` columns from the `edges.csv` table.

In [6]:
import dgl

src = edges_data['Src'].to_numpy()
dst = edges_data['Dst'].to_numpy()

# Create a DGL graph from a pair of numpy arrays
g = dgl.graph((src, dst))

# Print a graph gives some meta information such as number of nodes and edges.
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={}
      edata_schemes={})


A DGL graph can be converted to a `networkx` graph, so to utilize its rich functionalities such as visualization.

In [ ]:
import networkx as nx
# Since the actual graph is undirected, we convert it for visualization
# purpose.
nx_g = g.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_g)
nx.draw(nx_g, pos, with_labels=True, node_color=[[.7, .7, .7]])

## Query graph structures

Let's print out how many nodes and edges are there in this graph.

In [7]:
print('#Nodes', g.number_of_nodes())
print('#Edges', g.number_of_edges())

#Nodes 34
#Edges 156


We can also perform queries on the graph structures.

Get the in-degree of node 0:

In [8]:
g.in_degree(0)

16

Get the successors of node 0:

In [9]:
g.successors(0)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31])

DGL provides APIs for querying structural information. See the API document [here](https://docs.dgl.ai/api/python/heterograph.html#querying-graph-structure).

![image.png](../asset/dgl-query.png)

## Load node and edge features

In many graph data, nodes and edges have attributes. Although these attributes can have arbitrary types, a DGL graph only accepts attributes stored in tensors (with numerical contents). The vast development of deep learning has provided us many ways to vectorize various types of attributes into numerical features. Here are some general suggestions:
* For categorical attributes (e.g. gender, occupation), consider converting them to integers or one-hot encoding.
* For variable length string contents (e.g. news article, quote), consider applying a language model.
* For images, consider applying a vision model such as CNNs.

Our data set has the following attribute columns:
* `Age` is already an integer attribute.
* `Club` is a categorical attribute representing which community each member belongs to.
* `Weight` is a floating number indicating the strength of each interaction.

In [10]:
import torch
import torch.nn.functional as F

# Prepare the age node feature
age = torch.tensor(nodes_data['Age'].to_numpy()).float() / 100
print(age)

tensor([0.4500, 0.3300, 0.3600, 0.3100, 0.4100, 0.4200, 0.4800, 0.4100, 0.3000,
        0.3500, 0.3800, 0.4400, 0.3700, 0.3900, 0.3600, 0.3800, 0.4700, 0.4500,
        0.4100, 0.3100, 0.3100, 0.4400, 0.4200, 0.3200, 0.3000, 0.5000, 0.3000,
        0.4300, 0.4800, 0.4000, 0.3900, 0.4500, 0.4700, 0.3300])


In [22]:
torch.tensor(nodes_data['Age'].to_numpy()).float()/100

tensor([0.4500, 0.3300, 0.3600, 0.3100, 0.4100, 0.4200, 0.4800, 0.4100, 0.3000,
        0.3500, 0.3800, 0.4400, 0.3700, 0.3900, 0.3600, 0.3800, 0.4700, 0.4500,
        0.4100, 0.3100, 0.3100, 0.4400, 0.4200, 0.3200, 0.3000, 0.5000, 0.3000,
        0.4300, 0.4800, 0.4000, 0.3900, 0.4500, 0.4700, 0.3300])

In [11]:
# Get the features of node 0 and 10
age[[0, 10]]

tensor([0.4500, 0.3800])

Use `g.ndata` to set the age features to the graph.

In [23]:
# Feed the features to graph
g.ndata['age'] = age
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64), 'age': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})


In [13]:
# The "Club" column represents which community does each node belong to.
# The values are of string type, so we must convert it to either categorical
# integer values or one-hot encoding.

club = nodes_data['Club'].to_list()
# Convert to categorical integer values with 0 for 'Mr. Hi', 1 for 'Officer'.
club = torch.tensor([c == 'Officer' for c in club]).long()
# We can also convert it to one-hot encoding.
club_onehot = F.one_hot(club)
print(club_onehot)

# Use `g.ndata` like a normal dictionary
g.ndata.update({'club' : club, 'club_onehot' : club_onehot})
# Remove some features using del
del g.ndata['age']

print(g)

tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]])
Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


Feeding edge features to a DGL graph is similar.

In [14]:
# Get edge features from the DataFrame and feed it to graph.
edge_weight = torch.tensor(edges_data['Weight'].to_numpy())
# Similarly, use `g.edata` for getting/setting edge features.
g.edata['weight'] = edge_weight
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
